In [15]:
import os, pprint, csv, json, difflib
import textdistance as td
path_data = "/data/cm/"
path = "/home/rovera/cm/"

In [34]:
ann_set_1 = open(path_data+"data/ocr_correction/complete/annotation_set_1.csv")
ann_set_1 = list(csv.reader(ann_set_1, delimiter="\t"))
ann_set_2 = open(path_data+"data/ocr_correction/complete/annotation_set_2.csv")
ann_set_2 = list(csv.reader(ann_set_2, delimiter="\t"))
ann_set = ann_set_1 + ann_set_2
print(len(ann_set))

30331


In [38]:
def count_annotations(annotation_set):
    c = 0
    for line in annotation_set:
        if line[1] != "" and line[0] != line[1]:
            c += 1
    print("Annotated", c, "words.")

In [150]:
def viz_annotations(annotation_set):
    c = 0
    for line in annotation_set:
        if line[1] != '' and line[0] != line[1]:
            print(line[0], '\t', line[1])
            c += 1
            if c >= 10:
                break

In [10]:
viz_annotations(ann_set)

NameError: name 'viz_annotations' is not defined

In [37]:
count_annotations(ann_set)

Annotated 2048 words.


In [39]:
def get_difference(str1, str2):
    """
    Input: wrong string, correct string
    Output: deltas for the input strings, at character level
    """
    return list(difflib.ndiff(str1, str2))

In [40]:
def get_word_errors(diff):
    """
    Collect errors in the whole annotation set
    Input: output of get_difference
    Output: list of dicts, each dict contains: wrong substring, correct substring, wrong word, correct word
    """
    errors = []
    wr, co = "", ""
    in_err = False
    
    for char in diff:
        
        if char.startswith('+'):
            
            if in_err is False:
                in_err = True
            co += char[-1]
            
            if diff.index(char) == len(diff)-1:
                errors.append((wr, co))
            
        elif char.startswith('-'):
            
            if in_err is False:
                in_err = True
            wr += char[-1]
            
            if diff.index(char) == len(diff)-1:
                errors.append((wr, co))
            
        elif char.startswith(' '):
            if in_err is True:
                errors.append((wr, co))
                wr, co = "", ""
                in_err = False
        
    return errors

In [41]:
def get_error_patterns(all_errors):
    
    err_patterns = {}
    
    for error in all_errors:
        patt = (error['wr'], error['co'])
        example = (error['pred'], error['gold'])
        
        if patt not in err_patterns:
            err_patterns[patt] = []
        err_patterns[patt].append(example)
        
    return err_patterns

In [44]:
all_errors = []

for annotation in ann_set:
    if annotation[1] != "" and annotation[0] != annotation[1]: 
        
        diff = get_difference(annotation[0], annotation[1])
    
        errors = get_word_errors(diff)
        
        for e in errors:
            curr_err = {}
            curr_err['wr'] = e[0]
            curr_err['co'] = e[1]
            curr_err['pred'] = annotation[0]
            curr_err['gold'] = annotation[1]
            all_errors.append(curr_err)
print(len(all_errors))
err_patterns = get_error_patterns(all_errors)
err_patterns_sorted = sorted([(k, len(v)) for k,v in err_patterns.items()], key=lambda x:x[1], reverse=True)
print(len(err_patterns_sorted))

11264
3533


In [61]:
# print all error patterns with an example
all_err_patterns = []
for ep, examples in err_patterns.items():
    all_err_patterns.append([ep[0], ep[1], len(examples), examples[0][0], examples[0][1]])

all_err_patterns = sorted(all_err_patterns, key=lambda x:x[2], reverse=True)
#pprint.pprint(all_err_patterns)
with open(path_data+'data/ocr_correction/ocr_error_report_7500.csv', 'w', encoding="utf-8") as outfile:
    writer = csv.writer(outfile, delimiter='\t')
    for row in all_err_patterns:
        writer.writerow(row)

#for err_patt in all_err_patterns:
#    print(err_patt, err_patt[2]/11264*100)

In [60]:
for err_patt in all_err_patterns:
    if 'i' in err_patt[0] and 'l' in err_patt[1]:
        print(err_patt)

['i', 'l', 60, 'aliergnädigst', 'allergnädigst']
['hi', 'lu', 2, 'Chahizim', 'Chaluzim']
['bi', 'hl', 1, 'zabireicktc', 'zahlreiche']
['cit', 'dl', 1, 'Lekwinciterbanäe', 'Schwindlerbande']
['if', 'll', 1, 'gesteift', 'gestellt']
['stidi', 'lubl', 1, 'Kstidiokol', 'Klublokal']
['ie', 'la', 1, 'Pienetarium', 'Planetarium']
['i', 'ld', 1, 'Sehiieru', 'Schildern']
['iM', 'atl', 1, 'HsmiMosigkit', 'Heimatlosigkeit']
['si', 'fl', 1, 'unbcgreisiich', 'unbegreiflich']
['Äi', 'Alexand', 1, 'Äieranoer', 'Alexander']
['oi', 'al', 1, 'glsboid', 'alsbald']
['fi', 'l', 1, 'Pfiege', 'Plege']


In [56]:
for error in all_errors:
    if error['wr'] == "b" and error['co'] == 'h':
        print(error)

{'wr': 'b', 'co': 'h', 'pred': 'Nikiscbplatz', 'gold': 'Nikischplatz'}
{'wr': 'b', 'co': 'h', 'pred': 'zufübrende', 'gold': 'zuführende'}
{'wr': 'b', 'co': 'h', 'pred': 'Matbias', 'gold': 'Mathias'}
{'wr': 'b', 'co': 'h', 'pred': 'Racenbetze', 'gold': 'Racenhetze'}
{'wr': 'b', 'co': 'h', 'pred': 'abgebalte', 'gold': 'abgehalten'}
{'wr': 'b', 'co': 'h', 'pred': 'ibrer', 'gold': 'ihrer'}
{'wr': 'b', 'co': 'h', 'pred': 'wobin', 'gold': 'wohin'}
{'wr': 'b', 'co': 'h', 'pred': 'Beschaffenbeit', 'gold': 'Beschaffenheit'}
{'wr': 'b', 'co': 'h', 'pred': 'Bensbeim', 'gold': 'Bensheim'}
{'wr': 'b', 'co': 'h', 'pred': 'Tagficbt', 'gold': 'Taglicht'}
{'wr': 'b', 'co': 'h', 'pred': 'Faschfegsnacbt', 'gold': 'Faschingsnacht'}
{'wr': 'b', 'co': 'h', 'pred': 'Tonbtütbe', 'gold': 'Tonblüthe'}
{'wr': 'b', 'co': 'h', 'pred': 'abbing', 'gold': 'abhing'}
{'wr': 'b', 'co': 'h', 'pred': 'stebe', 'gold': 'stehen'}
{'wr': 'b', 'co': 'h', 'pred': 'febrt', 'gold': 'kehrt'}
{'wr': 'b', 'co': 'h', 'pred': 'katboli

In [58]:
c = 0
for k,v in err_patterns.items():
    if len(v)>= 2:
        c += 1
print(c)

748


In [62]:
def get_pattern_by_char(char, err_patterns):
    """
    Given a character or a small n-gram, returns, if existing, a list of patterns
    where the given char is the correct one 
    """
    
    char_patterns = {}
    
    for k,v in err_patterns.items():
        if k[1] == char:
            char_patterns[k] = len(v)
            
    print("Found", len(char_patterns), "patterns for character(s)", char)
    
    return char_patterns

In [63]:
l_patterns = get_pattern_by_char("E", err_patterns)
pprint.pprint(l_patterns)

Found 12 patterns for character(s) E
{('', 'E'): 1,
 ('B', 'E'): 2,
 ('C', 'E'): 4,
 ('F', 'E'): 3,
 ('G', 'E'): 6,
 ('K', 'E'): 9,
 ('L', 'E'): 2,
 ('Q', 'E'): 1,
 ('S', 'E'): 1,
 ('k', 'E'): 1,
 ('li', 'E'): 1,
 ('v', 'E'): 1}


In [65]:
# get two-way error patterns

two_way = []
for patt, examples in err_patterns.items():
    if (patt[1], patt[0]) in err_patterns.keys():
        count_reverse = len(err_patterns[(patt[1], patt[0])])
        two_way.append([patt, (patt[1], patt[0]), len(examples)+count_reverse])
    else:
        two_way.append([patt, (patt[1], patt[0]), len(examples)])
two_way_sorted = sorted(two_way, key=lambda x:x[2], reverse=True)
#pprint.pprint(two_way_sorted)
for patt in two_way_sorted:
    print(patt, patt[2]/11264*100)

[('c', 'e'), ('e', 'c'), 948] 8.416193181818182
[('e', 'c'), ('c', 'e'), 948] 8.416193181818182
[('u', 'n'), ('n', 'u'), 483] 4.287997159090909
[('n', 'u'), ('u', 'n'), 483] 4.287997159090909
[('f', 's'), ('s', 'f'), 372] 3.3025568181818183
[('s', 'f'), ('f', 's'), 372] 3.3025568181818183
[('', ' '), (' ', ''), 253] 2.24609375
[(' ', ''), ('', ' '), 253] 2.24609375
[('S', 's'), ('s', 'S'), 174] 1.5447443181818181
[('s', 'S'), ('S', 's'), 174] 1.5447443181818181
[('-', ''), ('', '-'), 167] 1.4825994318181819
[('', '-'), ('-', ''), 167] 1.4825994318181819
[('t', 'i'), ('i', 't'), 164] 1.455965909090909
[('i', 't'), ('t', 'i'), 164] 1.455965909090909
[('t', 'l'), ('l', 't'), 163] 1.4470880681818181
[('l', 't'), ('t', 'l'), 163] 1.4470880681818181
[('t', 'r'), ('r', 't'), 155] 1.376065340909091
[('r', 't'), ('t', 'r'), 155] 1.376065340909091
[('i', 'l'), ('l', 'i'), 140] 1.2428977272727273
[('l', 'i'), ('i', 'l'), 140] 1.2428977272727273
[('.', ''), ('', '.'), 133] 1.180752840909091
[('', 

[('P', 'S'), ('S', 'P'), 2] 0.01775568181818182
[('H', 'ch'), ('ch', 'H'), 2] 0.01775568181818182
[('d', 'k'), ('k', 'd'), 2] 0.01775568181818182
[('A', 'kl'), ('kl', 'A'), 2] 0.01775568181818182
[('l', 'j'), ('j', 'l'), 2] 0.01775568181818182
[('o', 'b'), ('b', 'o'), 2] 0.01775568181818182
[('', 'ng'), ('ng', ''), 2] 0.01775568181818182
[('ll', 'n'), ('n', 'll'), 2] 0.01775568181818182
[('v', 'i'), ('i', 'v'), 2] 0.01775568181818182
[('Ä', 'a'), ('a', 'Ä'), 2] 0.01775568181818182
[('', 'el'), ('el', ''), 2] 0.01775568181818182
[('ö', ''), ('', 'ö'), 2] 0.01775568181818182
[('vf', 'rs'), ('rs', 'vf'), 2] 0.01775568181818182
[('B', 'b'), ('b', 'B'), 2] 0.01775568181818182
[('M', 'hl'), ('hl', 'M'), 2] 0.01775568181818182
[('l', ' '), (' ', 'l'), 2] 0.01775568181818182
[('st', 'r'), ('r', 'st'), 2] 0.01775568181818182
[('L', 'k'), ('k', 'L'), 2] 0.01775568181818182
[('V', ''), ('', 'V'), 2] 0.01775568181818182
[('w', 'n '), ('n ', 'w'), 2] 0.01775568181818182
[('ll', 'tt'), ('tt', 'll'),

[('aff', 's'), ('s', 'aff'), 1] 0.00887784090909091
[('äirl', 'an'), ('an', 'äirl'), 1] 0.00887784090909091
[('ori', 'ent'), ('ent', 'ori'), 1] 0.00887784090909091
[('-ffou', 'ssen'), ('ssen', '-ffou'), 1] 0.00887784090909091
[('tte', ''), ('', 'tte'), 1] 0.00887784090909091
[('äv', 'ar'), ('ar', 'äv'), 1] 0.00887784090909091
[('Mm', 'i'), ('i', 'Mm'), 1] 0.00887784090909091
[('', 'wil'), ('wil', ''), 1] 0.00887784090909091
[('b', ' d'), (' d', 'b'), 1] 0.00887784090909091
[("'-ü'", 'u'), ('u', "'-ü'"), 1] 0.00887784090909091
[('av', 'n'), ('n', 'av'), 1] 0.00887784090909091
[('wÄ.', 'nd'), ('nd', 'wÄ.'), 1] 0.00887784090909091
[('-o', 'e'), ('e', '-o'), 1] 0.00887784090909091
[("»;.'", 'n '), ('n ', "»;.'"), 1] 0.00887784090909091
[('smr', 'fon'), ('fon', 'smr'), 1] 0.00887784090909091
[('', 'du'), ('du', ''), 1] 0.00887784090909091
[('s', 'p'), ('p', 's'), 1] 0.00887784090909091
[('hp', 'ig'), ('ig', 'hp'), 1] 0.00887784090909091
[('', 'rno'), ('rno', ''), 1] 0.00887784090909091
[('M

[("fliC'", 'üts-'), ('üts-', "fliC'"), 1] 0.00887784090909091
[('&', 'ö'), ('ö', '&'), 1] 0.00887784090909091
[('di', 'sch'), ('sch', 'di'), 1] 0.00887784090909091
[('difi', 'chü'), ('chü', 'difi'), 1] 0.00887784090909091
[('o', 'ei'), ('ei', 'o'), 1] 0.00887784090909091
[('M', 'kü'), ('kü', 'M'), 1] 0.00887784090909091
[('lrv', '50'), ('50', 'lrv'), 1] 0.00887784090909091
[('s', 'ei'), ('ei', 's'), 1] 0.00887784090909091
[('iM', 'atl'), ('atl', 'iM'), 1] 0.00887784090909091
[('sM', 'ch'), ('ch', 'sM'), 1] 0.00887784090909091
[('-D', 'S'), ('S', '-D'), 1] 0.00887784090909091
[('ito', 'be'), ('be', 'ito'), 1] 0.00887784090909091
[('M', 'Si'), ('Si', 'M'), 1] 0.00887784090909091
[('M', 'G'), ('G', 'M'), 1] 0.00887784090909091
[('to', 'da'), ('da', 'to'), 1] 0.00887784090909091
[('nP', 'me'), ('me', 'nP'), 1] 0.00887784090909091
[('W', 'schl'), ('schl', 'W'), 1] 0.00887784090909091
[("l'", 'i'), ('i', "l'"), 1] 0.00887784090909091
[('KMq', 'Redac'), ('Redac', 'KMq'), 1] 0.0088778409090909

In [68]:
# get most difficult charcters to predict (by number of errors)
most_difficult = {}
for k,v in err_patterns.items():
    if len(k[1]) == 2:
        if k[1] not in most_difficult:
            most_difficult[k[1]] = 0
        most_difficult[k[1]] += len(v)
        
    
most_difficult = sorted([(k,v) for k,v in most_difficult.items()], key=lambda x:x[1], reverse=True)
pprint.pprint(most_difficult)

[('en', 112),
 ('ss', 96),
 ('- ', 93),
 ('ch', 74),
 ('s ', 69),
 (', ', 43),
 ('in', 41),
 ('un', 30),
 ('te', 28),
 ('er', 27),
 ('nd', 23),
 ('st', 23),
 ('mm', 23),
 ('ll', 21),
 ('me', 20),
 ('li', 19),
 ('es', 19),
 ('ei', 17),
 ('se', 17),
 ('an', 16),
 ('ni', 16),
 ('he', 16),
 ('au', 15),
 ('ri', 14),
 ('. ', 13),
 ('n ', 13),
 ('on', 12),
 ('ff', 12),
 ('be', 12),
 ('nt', 12),
 ('is', 12),
 ('de', 12),
 (' d', 11),
 ('ns', 11),
 ('il', 10),
 ('ie', 10),
 ('e ', 9),
 ('ra', 9),
 ('Re', 9),
 ('us', 9),
 ('uf', 9),
 ('fe', 9),
 ('tz', 9),
 (' z', 9),
 ('ge', 8),
 ('em', 8),
 ('el', 8),
 ('rn', 8),
 ('sc', 8),
 ('ru', 8),
 ('Au', 7),
 ('si', 7),
 ('ck', 7),
 ('tt', 7),
 ('Ge', 7),
 ('ha', 7),
 ('re', 7),
 ('ne', 7),
 ('Is', 7),
 ('ng', 6),
 ('ts', 6),
 ('it', 6),
 ('Ju', 6),
 ('ul', 6),
 ('zt', 6),
 ('rm', 6),
 ('nf', 6),
 ('fl', 6),
 ('ti', 6),
 ('Be', 6),
 (' i', 6),
 ('r ', 6),
 ('rt', 5),
 ('le', 5),
 ('Ab', 5),
 ('De', 5),
 ('rs', 5),
 ('so', 5),
 ('ue', 5),
 ('Si', 5),
 ('

In [187]:
def compute_text_distance(str1, str2):
    distances = {}
    distances['lev'] = td.levenshtein(str1, str2)
    distances['ham'] = td.hamming(str1, str2)
    distances['jac'] = td.jaccard(str1, str2)
    distances['jaro'] = td.jaro(str1, str2)
    distances['cos'] = td.cosine(str1, str2)
    return distances

In [192]:
distances = []
for line in ann_set:
    if line[1] != '' and line[0] != line[1]:
        distance = []
        dist = compute_text_distance(line[0], line[1])
        distance.append(line[0]+"__"+line[1])
        distance.append(dist['lev'])
        distance.append(dist['ham'])
        distance.append(dist['jac'])
        distance.append(dist['jaro'])
        distance.append(dist['cos'])
        distances.append(distance)
print(len(distances))       
with open(path_data+'data/ocr_correction/word_distance.csv', 'w', encoding="utf-8") as outfile:
    writer = csv.writer(outfile, delimiter='\t')
    for row in distances:
        writer.writerow(row)

2033
